In [1]:
import pandas as pd
from cStringIO import StringIO
import re
import os

In [2]:
def file_to_df(lines,my_file):

    re_pattern = r"\t\d{1,3}%\r\n"

    filtered_lines = [l for l in lines if re.search(re_pattern, l) and l.count("\t") > 4]

    filtered_lines = [l.replace("\t**", "") for l in filtered_lines]

    f = "\n".join(filtered_lines)

    f2 = StringIO(f)
 
    df = pd.read_csv(f2, sep = "\t", header=None)
    df.columns = ["prison_name","baseline_cna", "in_use_cna", "operational_capacity", "population", "perc_pop_to_used_cna", "perc_acc_available"]
    
    #Parse out the date here
    re_pattern = r"(\d{2}/\d{2}/\d{4})"
    
    f = " ".join(lines)
    date = re.search(re_pattern, f).group(1)   
    df["date"] = date
    df["source"] = my_file

    return df

In [3]:
# Now iterate through all the .docs which were converted in VBA
dfs = []
for i in os.listdir("all_doc_and_docx_to_txt/all_as_text/"):
    if i.endswith(".txt"): 
        with open("all_doc_and_docx_to_txt/all_as_text/"+i) as f:
            dfs.append(file_to_df(f.readlines(),i))
    else:
        continue

In [4]:
def df_to_numeric(df):
    df.iloc[:,1:8] = df.iloc[:,1:8].replace("%", "", regex=True).replace(",", "", regex=True).replace("\*", "", regex=True)

    for c in df.columns[1:8]:
        df[c]= pd.to_numeric(df[c], errors='ignore')
    cols = [c for c in df.columns if "perc" in c]
    df.loc[:,cols] = df.loc[:,cols]/100
    df["date"] = pd.to_datetime(df["date"],infer_datetime_format=True)
    df["prison_name"] = df["prison_name"].str.strip()
    return df

In [5]:
df_final = pd.concat(dfs)
df_final = df_to_numeric(df_final)

In [6]:
df_final.head()

,prison_name,baseline_cna,in_use_cna,operational_capacity,population,perc_pop_to_used_cna,perc_acc_available,date,source
0,Acklington,946,946,946,836,0.88,1.00,2011-01-28,10004C44pop_bull_jan_11.txt
1,Altcourse,794,794,1324,1173,1.48,1.00,2011-01-28,10004C44pop_bull_jan_11.txt
2,Ashfield,407,407,360,315,0.77,1.00,2011-01-28,10004C44pop_bull_jan_11.txt
3,Ashwell,599,184,214,159,0.86,0.31,2011-01-28,10004C44pop_bull_jan_11.txt
4,Askham Grange,150,126,128,128,1.02,0.84,2011-01-28,10004C44pop_bull_jan_11.txt


In [7]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

df_final.to_csv("all_pop_data_txt.csv", index=False, encoding="utf-8")